In [22]:
!pip install transformers

In [3]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

prompt = "Breaking news: scientists discovered a new virus in"
result = generator(prompt, max_length=100, num_return_sequences=1)

print("\n📢 Generated Fake News:\n")
print(result[0]['generated_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📢 Generated Fake News:

Breaking news: scientists discovered a new virus in a mosquito.

In a new study published in the journal Nature Communications, scientists found that human-caused Zika virus is the most effective candidate to treat the condition, which is characterized by a "long-term fever" and weak immune responses.

"This study shows we're on the right track," said lead author Dr. Brian Smith, a biomedical epidemiologist at the University of California, Santa Barbara.

The virus, known as DYNAS, is a highly pathogenic, multi-drug-resistant, pathogen that makes its way into the blood and infects people in about 15 states and territories, according to the Centers for Disease Control and Prevention.

The virus has been found in more than 50 people, and is one of the most common infections in the United States, according to CDC statistics.

DYNAS is also believed to be a key ingredient in the Zika virus, so it has been studied extensively, Smith said.

"We're excited to see how 

In [4]:
with open("generated_fake_news.txt", "w") as f:
    f.write(result[0]['generated_text'])

print("✅ Fake news saved to generated_fake_news.txt")

✅ Fake news saved to generated_fake_news.txt


In [7]:
!pip install pandas scikit-learn numpy

In [2]:
!pip install -U scikit-learn

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [4]:
import kagglehub

path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fake-and-real-news-dataset


In [7]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [8]:
import zipfile
import os

with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

os.listdir("data")

['True.csv', 'Fake.csv']

In [9]:
import pandas as pd

true_df = pd.read_csv("data/True.csv")
fake_df = pd.read_csv("data/Fake.csv")

print("True News:")
print(true_df.head())

print("\nFake News:")
print(fake_df.head())

True News:
                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   

Fake News:
                                               title  \
0   Donald Trump Se

In [10]:
true_df['label'] = 1
fake_df['label'] = 0

data = pd.concat([true_df, fake_df], ignore_index=True)

data = data.sample(frac=1).reset_index(drop=True)

data.head()

,title,text,subject,date,label
0,Biden: Sanders has 'struck a chord with voters...,WASHINGTON (Reuters) - U.S. Vice President Joe...,politicsNews,"January 13, 2016",1
1,ABSOLUTE SUBMISSION: Trump Bows to Neocon Orth...,Consortium News Exclusive: In his Mideast trip...,Middle-east,"June 1, 2017",0
2,BOYCOTT BACKFIRES! Ivanka Trump Clothing Line ...,Ivanka Trump s eponymous women s fashion line ...,politics,"Mar 7, 2017",0
3,The BRUTAL Truth About Why Kids Love Bernie Sa...,"At some point, most of us grow up and realize...",politics,"Mar 12, 2016",0
4,Mueller tells White House to save files on Don...,WASHINGTON (Reuters) - Special counsel Robert ...,politicsNews,"July 21, 2017",1


In [11]:
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

data['text'] = data['title'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9456570155902004

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.94      0.95      4741
           1       0.93      0.95      0.94      4239

    accuracy                           0.95      8980
   macro avg       0.95      0.95      0.95      8980
weighted avg       0.95      0.95      0.95      8980



In [13]:
data['label'].value_counts()

,count
label,
0,23481
1,21417


In [14]:
data['combined'] = data['title'] + " " + data['text']

data['combined'] = data['combined'].apply(clean_text)

X_train, X_test, y_train, y_test = train_test_split(data['combined'], data['label'], test_size=0.2, random_state=42)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

model = LogisticRegression()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9456570155902004


In [15]:
def predict_news(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    prediction = model.predict(vec)[0]
    return "Real News" if prediction == 1 else "Fake News"

print(predict_news("Donald Trump caught lying in public speech"))
print(predict_news("NASA discovers water on Mars"))

Fake News
Real News


In [16]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")

generated = generator("Breaking News: ", max_length=50, num_return_sequences=3)

for i, g in enumerate(generated):
    print(f"\nGenerated News {i+1}:\n", g['generated_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_t


Generated News 1:
 Breaking News:  We're getting to the middle of the night at 7:30 PM EST on Saturday.  I've got a report on it.  The first thing I did was to go to the National Endowment for the Arts and I got an email from the director of the National Endowment for the Arts saying that they received an email from a person who's been in government.  I went to my lawyer's office and said "Oh, I know you're doing something right."  The attorney went out and spoke to the director.  I said "I can't believe you sent this email.  I'm just glad that you called me."  The attorney said "Well, I was on the phone with the director and you were right.  I'm sorry, but the director is on the same team.  I have no idea what your email is about.  I don't want to get into a personal dispute with you.  I'm sure if it's something you're doing right now, I can understand your concerns.  I'm sorry, but you're not going to be able to get into a legal dispute about a government contract.  I'm just glad th